# Tarea 2
## Regresion Lineal y Logistica 

### DataSet
* dataset de vuelos de argentina
* url: https://datos.gob.ar/dataset/transporte-aterrizajes-despegues-procesados-por-administracion-nacional-aviacion-civil-anac
* Seleccionar solo un año


In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy as sym


In [11]:
argentina = pd.read_excel('Dataset/2023-2.xlsx')

argentina


,Fecha UTC,Hora UTC,Clase de Vuelo (todos los vuelos),Clasificación Vuelo,Tipo de Movimiento,Aeropuerto,Origen / Destino,Aerolinea Nombre,Aeronave,Pasajeros,PAX,Calidad dato
0,2023-01-01,00:01:00,Regular,Internacional,Aterrizaje,EZE,LEMD,IBERIA - LINEAS AÉREAS DE ESPAÑA,0,199,199,DEFINITIVO
1,2023-01-01,00:10:00,Regular,Doméstico,Despegue,IGU,EZE,JETSMART AIRLINES S.A.,AIB-A320-232,67,34,DEFINITIVO
2,2023-01-01,00:15:00,Regular,Internacional,Aterrizaje,AER,SBPA,AEROLINEAS ARGENTINAS SA,BO-B-737-76N,36,36,DEFINITIVO
3,2023-01-01,00:17:00,Regular,Doméstico,Aterrizaje,SAL,AER,JETSMART AIRLINES S.A.,AIB-A320-232,168,84,DEFINITIVO
4,2023-01-01,00:19:00,Regular,Doméstico,Aterrizaje,AER,OSA,AEROLINEAS ARGENTINAS SA,EMB-ERJ190100IGW,17,9,DEFINITIVO
...,...,...,...,...,...,...,...,...,...,...,...,...
557147,2023-12-31,23:45:00,Trabajo Aéreo,Doméstico,Aterrizaje,DOZ,SRA,0,PA-PA-31-T,0,0,DEFINITIVO
557148,2023-12-31,23:50:00,Regular,Doméstico,Despegue,AER,CBA,JETSMART AIRLINES S.A.,AIB-A320-232,174,87,DEFINITIVO
557149,2023-12-31,23:51:00,Regular,Doméstico,Despegue,NEU,AER,JETSMART AIRLINES S.A.,AIB-A320-232,143,72,DEFINITIVO
557150,2023-12-31,23:55:00,Regular,Doméstico,Despegue,AER,CBA,AEROLINEAS ARGENTINAS SA,BO-B737-800,166,83,DEFINITIVO


In [19]:
# Definir el orden de las categorías
orden = ['No Regular', 'Regular', 'Trabajo Aéreo', 'Vuelo de Adiestramiento', 'Vuelo Escuela', 'Vuelo Oficial Extranjero', 'Vuelo Oficial Nacional', 'Vuelo Privado con Matrícula Extranjera', 'Vuelo Privado con Matrícula Nacional']

# Mapear las categorías a números utilizando el nombre correcto de la columna
argentina['codificacion_clase_vuelos'] = argentina['Clase de Vuelo (todos los vuelos)'].map({cat: idx for idx, cat in enumerate(orden)})

argentina

        Fecha UTC  Hora UTC Clase de Vuelo (todos los vuelos)  \
18     2023-01-01  00:44:00                        No Regular   
42     2023-01-01  01:55:00                        No Regular   
103    2023-01-01  08:40:00                        No Regular   
126    2023-01-01  10:00:00                        No Regular   
165    2023-01-01  11:17:00                        No Regular   
...           ...       ...                               ...   
556942 2023-12-31  20:08:00                        No Regular   
556991 2023-12-31  20:53:00                        No Regular   
556994 2023-12-31  20:53:00                        No Regular   
557056 2023-12-31  21:58:00                        No Regular   
557085 2023-12-31  22:30:00                        No Regular   

       Clasificación Vuelo Tipo de Movimiento Aeropuerto Origen / Destino  \
18               Doméstico         Aterrizaje        FDO              TUC   
42           Internacional         Aterrizaje        EZE         

## Petitorio

* Realizar un analisis y descripcion de los datos
* Limpiar los datos de ser necesario
* Realizar una prediccion utilizandon la regresion lineal
* Realizar una prediccion o clasificacion con el metodo de regresion logistica
* Para las predicciones puede utilizar las variables que el equipo seleccione
##### Investigacion -> sobre tablas de contingencia 

### Presentar

* Presentar un informe con capturas y conclusiones de los hallazgos
* conclusiones sobre los modelos entrenados(score,tablas de contingencias,mejor porcentaje de Dataset de entrenamiento y pruebas)
* Formato .pdf, incluir los nombres de los integrantes del grupo
